In [115]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [116]:
import pandas as pd

import proyeccion_rdr.features as features
import proyeccion_rdr.produccion.a03_pabellon as pb

pd.set_option("display.max_columns", None)


ANIO_INICIO = 2017
ANIO_TERMINO = 2035
# Agrega las columnas de poblacion de interes
ANIOS_INTERES = [str(i) for i in range(2017, 2036)]

## Obtencion de casos por area de influencia INT

In [117]:
# Lee los casos de todos los macroprocesos
RUTA_ARCHIVOS = "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx"
_, casos_macroproceso = features.leer_casos_macroprocesos(RUTA_ARCHIVOS)

# Obtiene los casos quirurgicos
casos_quirurgicos = casos_macroproceso.query("tipo_paciente == 'quirurgicos'")

In [118]:
# Los diagnosticos que tienen un porcentaje de hosp. mayor a 0
diagnosticos_relevantes = list(casos_quirurgicos.index)

# Ademas, los diagnosticos que estan agrupados dentro de los que aportan a AC
# Solo deja los diagnosticos que esten agrupados
diagnosticos_a_reasignar = casos_quirurgicos.dropna(subset="Diagnosticos Contenidos")

# Separa la lista de diagnosticos en filas
diagnosticos_agrupados = diagnosticos_a_reasignar.explode("Diagnosticos Contenidos")
diagnosticos_agrupados = list(diagnosticos_agrupados["Diagnosticos Contenidos"].unique())

# Suma los diagnosticos relevantes y los agrupados
diagnosticos_a_buscar_en_quir = sorted(diagnosticos_relevantes + diagnosticos_agrupados)

## Obtencion de tiempos quirurgicos entre 2015 y 2022 en pabellon para diagnosticos relevantes

Estos se obtendran a partir de la union de la base de GRD (2015 a 2022, tiene los diagnosticos codificados) y Pabellon (2015 a 2022, tiene los tiempos quirurgicos).

In [119]:
# Lee la base de GRD unida con Pabellon
grd_y_pabellon = pd.read_csv("../data/raw/5_grd_y_pabellon/df_procesada_grd_y_pabellon.csv")
grd_y_pabellon["t_total"] = pd.to_timedelta(grd_y_pabellon["t_total"].str.split(" ").str[1])

C:\Users\javie\AppData\Local\Temp\ipykernel_34808\3808083235.py:2: DtypeWarning: Columns (41,42,43,44,45,46,47,48,49,50,51,52,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,149,150,151,152,153,154,155,156,157,158,230) have mixed types. Specify dtype option on import or set low_memory=False.
  grd_y_pabellon = pd.read_csv("../data/raw/5_grd_y_pabellon/df_procesada_grd_y_pabellon.csv")


In [120]:
# Filtra solamente los diagnosticos que aportan a quirurgico
grd_y_pabellon_relevantes = grd_y_pabellon.query(
    "diagnostico_1.isin(@diagnosticos_a_buscar_en_quir)"
).copy()

# Elimina las operaciones hechas en ambulatorio
grd_y_pabellon_relevantes = grd_y_pabellon_relevantes.query("tipo_cirugia != 'AMBULATORIO'")

In [121]:
# Lee los egresos del Torax
egresos_rdr = pd.read_csv("../data/raw/4_egresos_deis/egresos_procesados_109101.csv")

C:\Users\javie\AppData\Local\Temp\ipykernel_34808\3328051442.py:2: DtypeWarning: Columns (0,7,11,13,33,36) have mixed types. Specify dtype option on import or set low_memory=False.
  egresos_rdr = pd.read_csv("../data/raw/4_egresos_deis/egresos_procesados_109101.csv")


In [122]:
# Obtiene los egresos quirurgicos desde DEIS
egresos_rdr_iq = egresos_rdr.query("INTERV_Q == 1")

conteo_egresos_rdr_iq = egresos_rdr_iq.groupby("ANO_EGRESO").size()
conteo_egresos_rdr_iq_relevantes = egresos_rdr_iq_relevantes = (
    egresos_rdr_iq.query("DIAG1.isin(@diagnosticos_a_buscar_en_quir)").groupby("ANO_EGRESO").size()
)
comparacion_iq = pd.concat([conteo_egresos_rdr_iq, conteo_egresos_rdr_iq_relevantes], axis=1)
comparacion_iq["%"] = round((comparacion_iq[1] / comparacion_iq[0]) * 100, 2)

In [123]:
comparacion_iq

,0,1,%
ANO_EGRESO,,,
2001.0,4618,627,13.58
2002.0,4721,600,12.71
2003.0,4462,677,15.17
2004.0,4358,640,14.69
2005.0,3597,346,9.62
2006.0,1581,69,4.36
2007.0,3498,369,10.55
2008.0,895,79,8.83
2009.0,1497,156,10.42


## Reasginar diagnosticos

In [124]:
for row in diagnosticos_a_reasignar.itertuples():
    diagnostico_nuevo = row[0]
    diagnosticos_antiguos = row[1]

    print(f"Cambiando {diagnosticos_antiguos} a {diagnostico_nuevo}")
    diagnosticos_cambiados = grd_y_pabellon_relevantes["diagnostico_1"].replace(
        diagnosticos_antiguos, diagnostico_nuevo
    )
    grd_y_pabellon_relevantes["diagnostico_1"] = diagnosticos_cambiados

Cambiando ['Q262', 'K914', 'K638'] a ENTN
Cambiando ['H652', 'H653', 'H654', 'H669'] a HOTI
Cambiando ['J121', 'J122', 'J123', 'J129', 'J159', 'J189'] a JNEU
Cambiando ['K352', 'K353', 'K358', 'K36X', 'K659'] a KAPE
Cambiando ['K565', 'K650'] a KGAST
Cambiando ['M411', 'M410', 'M412', 'M414', 'T813', 'Q763'] a MESC
Cambiando ['K560', 'Q625', 'T222'] a OBSI
Cambiando ['Q390', 'Q391', 'Q393'] a QATR
Cambiando ['Q375', 'Q374', 'Q369', 'Q371', 'Q379', 'Q373', 'Q370', 'T813', 'Q355', 'Q372', 'Q359', 'Q378'] a QLAB
Cambiando ['S424', 'S524', 'S525', 'S528', 'S529'] a SFRA
Cambiando ['T213', 'T202', 'T212', 'T203', 'T293', 'T201', 'T273', 'T223', 'T222', 'T243', 'T280', 'T211', 'T242', 'T253', 'T232', 'T221', 'T241', 'T312', 'T291', 'T292', 'T252'] a TQUE
Cambiando ['C710', 'C715', 'C716', 'C717', 'C719', 'D164', 'D334', 'D431', 'D444'] a CERX


## Obtener tiempos quirurgicos

In [125]:
# Obtiene el resumen de duraciones de las int. q por diagnostico acumuladas
tiempos_quir = pd.to_timedelta(
    grd_y_pabellon_relevantes.groupby(["diagnostico_1"])["t_total"].describe()["75%"]
)
tiempos_quir.loc["TQUED"] = tiempos_quir.loc["TQUE"]

In [126]:
# Obtiene la cantidad de intervenciones quirurgicas por cada tipo de paciente
conteo_int_q_por_paciente = (
    grd_y_pabellon_relevantes.groupby(["ano_x", "diagnostico_1", "id_paciente"])
    .agg(n_int_q=("fecha_operacion", "count"))
    .reset_index()
    .groupby(["ano_x", "diagnostico_1"])["n_int_q"].describe()
)

# Selecciona cuantas intervenciones quirurgicas tendra cada paciente segun el diagnostico
numero_int_q_por_pacientes = conteo_int_q_por_paciente.query("ano_x == 2024")["75%"]
numero_int_q_por_pacientes = numero_int_q_por_pacientes.reset_index(level=0, drop=True)

# Agrega Numero de Intervenciones quirurgicas para Q410 del 2021 (No Habia para el 2024)
numero_int_q_por_pacientes.loc["Q410"] = 4
numero_int_q_por_pacientes.loc["TQUED"] = numero_int_q_por_pacientes.loc["TQUE"]

In [127]:
# Calcula la cantidad de intervenciones quirurgicas que se realizaran
numero_de_int_q = pb.calcular_cantidad_int_q(
    casos_quirurgicos[ANIOS_INTERES],
    numero_int_q_por_pacientes,
)

# Calcular tiempo utilizado en pabellón en horas
tiempo_utilizado_pabellon_horas = pb.calcular_tiempo_utilizado_pabellon(
    numero_de_int_q,
    tiempos_quir,
)

# Calcular horas laborales
horas_laborales = features.calcular_horas_laborales(ANIO_INICIO, ANIO_TERMINO, 12)

# Calcular cantidad de pabellones necesarios
cantidad_de_pabellones_necesarios = pb.calcular_cantidad_de_pabellones_necesarios(
    tiempo_utilizado_pabellon_horas, horas_laborales
)

# Obtiene la suma total de pabellones
suma_total_pabellones = cantidad_de_pabellones_necesarios.sum()

Cantidad de Int. Q por diagnostico:
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+
|      |        2017        |        2018        |        2019        |        2020        |        2021        |        2022        |       2023        |        2024        |        2025        |        2026        |        2027        |        2028        |        2029        |        2030        |       2031        |        2032        |        2033        |        2034        |       2035        |
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+

In [128]:
print(f"> Pabellones 2035: {suma_total_pabellones['2035']:.2f}")

> Pabellones 2035: 7.05


In [142]:
# Ademas, sabemos que estas trazadoras cubren el 65% de la demanda
# Y mencionaremos que para el 2035 estos diags cubriran el 75% de la demanda
# Por lo tanto, existe un resto que permite proyectar todo
suma_total_pabellones_con_resto = suma_total_pabellones / 0.75

In [143]:
print(f"> Pabellones 2035 + Resto 2035: {suma_total_pabellones_con_resto['2035']:.2f}")

> Pabellones 2035 + Resto 2035: 9.40


## Pabellon de Emergencia

In [131]:
# Obtiene los casos por especialidad por anio
casos_especialidad_desglosado = casos_macroproceso.query("tipo_paciente == 'Placeholder'")
casos_especialidad = casos_especialidad_desglosado.groupby("tipo_paciente")[ANIOS_INTERES].sum()

## Obtener resumen para MINSAL

In [132]:
# Obtiene porcentaje de hospitalizados
porcentaje_cm = pd.read_excel(
    "../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_RDR.xlsx"
)

porcentaje_cm["Diagnostico"] = porcentaje_cm["Diagnostico"].str.split(" - ").str[0]

# Filtra los porcentajes de hospitalizados
porcentaje_cm = porcentaje_cm.set_index("Diagnostico")
porcentaje_cm = porcentaje_cm["Porcentaje Pacientes Quirúrgicos"]
porcentaje_cm = porcentaje_cm.dropna()
porcentaje_cm = porcentaje_cm[porcentaje_cm > 0]

In [133]:
# Carga el resumen de area de estudio para MINSAL
resumen_area_de_estudio_minsal = pd.read_excel(
    "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx", sheet_name="resumen_total_RDR"
).sort_values("Diagnostico")
resumen_area_de_estudio_minsal["diagnostico_separado"] = (
    resumen_area_de_estudio_minsal["Diagnostico"].str.split(" - ").str[0]
)
resumen_area_de_estudio_minsal = resumen_area_de_estudio_minsal.set_index("diagnostico_separado")

# Sintetiza informacion de pabellon para el 2035
resumen_quirurgicos = pd.DataFrame(
    {
        "especialidad_quirurgica": casos_especialidad_desglosado["tipo_paciente"],
        "porcentaje_quirurgico_cm": porcentaje_cm,
        "casos_quirurgicos_2035": casos_quirurgicos["2035"],
        "n_int_q_por_paciente_75%": numero_int_q_por_pacientes,
        "n_int_q_2035": numero_de_int_q["2035"],
        "tiempo_quirurgico_75%": tiempos_quir,
        "horas_pabellon_2035": tiempo_utilizado_pabellon_horas["2035"],
        "horas_laborales_2035_pabellon_12_hrs": horas_laborales["2035"],
        "cantidad_de_pabellones": cantidad_de_pabellones_necesarios["2035"],
    }
)

# Une ambos resumenes segun el diagnostico
resumen_minsal = resumen_area_de_estudio_minsal.merge(
    resumen_quirurgicos, how="inner", left_index=True, right_index=True
)

In [134]:
resumen_minsal

,Diagnostico,Diagnosticos Contenidos,Diagnostico agregado en el macroproceso,Estadística,Casos (Cada 100.000),Edad Incidencia,poblacion_ine_2035_area_de_estudio,casos_teoricos_ine_2035_area_de_estudio,porcentaje_fonasa_pais,casos_teoricos_fonasa_2035_area_de_estudio,Área de Influencia Formal,Área de Influencia Propuesta,casos_teoricos_fonasa_2035_area_de_influencia,Casos a hacerse cargo del Área de Influencia Propuesta,casos_teoricos_fonasa_2035_a_hacerse_cargo,especialidad_quirurgica,porcentaje_quirurgico_cm,casos_quirurgicos_2035,n_int_q_por_paciente_75%,n_int_q_2035,tiempo_quirurgico_75%,horas_pabellon_2035,horas_laborales_2035_pabellon_12_hrs,cantidad_de_pabellones
CERX,CERX - Tumores cerebrales malignos y benignos,"C710, C715, C716, C717, C719, D164, D334, D431...",2 - Quirúrgico,Incidencia,5.400,entre_0_y_18,4356495,235.250730,0.7477,175.896983,NaN,SSMN,11.213043,1.00,11.213043,NaN,0.9000,10.091738,3.00,30.275215,0 days 08:56:00,270.458591,2952,0.091619
ENTN,ENTN - Enterocolitis Necrotizante del Feto,"Q262, K914, K638",2.1 - Quirúrgico Neonatal,Incidencia,250.000,recien_nacidos_vivos,212799,531.997500,0.7477,397.774557,NaN,SSMN,26.028636,1.00,26.028636,NaN,0.4000,10.411454,4.50,46.851545,0 days 05:00:00,234.257724,2952,0.079356
HOTI,HOTI - Otitis media cronica,"H652, H653, H654, H669",2 - Quirúrgico,Prevalencia,75000.000,hasta_10,2459074,368861.100000,0.7477,275797.462851,NaN,SSMN,18111.498333,1.00,18111.498333,NaN,0.0075,135.836237,1.00,135.836237,0 days 03:22:30,458.447302,2952,0.155301
J955,J955 - Estenosis subglótica consecutiva a proc...,NaN,2 - Quirúrgico,Incidencia,0.225,entre_0_y_18,4356495,9.802114,0.7477,7.329041,NaN,SSMN,0.467210,1.00,0.467210,NaN,1.0000,0.467210,4.00,1.868840,0 days 02:10:00,4.049154,2952,0.001372
JNEU,JNEU - Neumonías,"J121, J122, J123, J129, J159, J189",1 - Atención Cerrada,Incidencia,138.500,entre_0_y_18,4356495,6033.745575,0.7477,4511.431867,NaN,SSMN,287.593782,1.00,287.593782,NaN,0.0500,14.379689,1.00,14.379689,0 days 02:14:15,32.174554,2952,0.010899
KAPE,KAPE - Apendicitis,"K352, K353, K358, K36X, K659",1 - Atención Cerrada,Incidencia,210.000,entre_0_y_18,4356495,9148.639500,0.7477,6840.438210,NaN,SSMN,436.062774,1.00,436.062774,NaN,1.0000,436.062774,1.00,436.062774,0 days 02:45:00,1199.172629,2952,0.406224
KGAST,KGAST - Gastrosquisis,"K565, K650",2.1 - Quirúrgico Neonatal,Incidencia,50.000,recien_nacidos_vivos,212799,106.399500,0.7477,79.554911,NaN,"SSMN, Coquimbo, Atacama",10.641821,1.00,10.641821,NaN,1.0000,10.641821,1.50,15.962732,0 days 03:48:30,60.791404,2952,0.020593
MESC,MESC - Escoliosis idiopática,"M411, M410, M412, M414, T813, Q763",2 - Quirúrgico,Prevalencia,3830.000,entre_10_y_18,2132116,16332.008560,0.7477,12211.443614,NaN,"SSMN, Atacama, SSMS, Coquimbo, Biobío",3519.437403,1.00,3519.437403,NaN,0.1000,351.943740,1.00,351.943740,0 days 07:38:00,2686.503884,2952,0.910062
N390,"N390 - Infección de vías urinarias, sitio no e...",NaN,1 - Atención Cerrada,Prevalencia,6667.000,entre_0_y_18,4356495,58089.504330,0.7477,43433.525282,NaN,SSMN,2768.790967,1.00,2768.790967,NaN,0.0100,27.687910,1.00,27.687910,0 days 01:55:00,53.068494,2952,0.017977
OBSI,OBSI - Obstrucción Intestinal del recién nacido,"K560, Q625, T222",2.1 - Quirúrgico Neonatal,Incidencia,66.670,recien_nacidos_vivos,212799,141.873093,0.7477,106.078519,NaN,"SSMN, Coquimbo, Atacama",14.189804,1.00,14.189804,NaN,1.0000,14.189804,1.50,21.284707,0 days 03:20:30,71.126395,2952,0.024094


In [135]:
a_guardar = {
    "resumen_MINSAL": resumen_minsal,
    "casos_quirurgicos": casos_especialidad_desglosado,
    "casos_por_especialidad": casos_especialidad,
    # "casos_complicaciones": casos_complicaciones,
    "resumen_duraciones_int_q_rel": tiempos_quir,
    "tiempo_utilizado_pabellon": tiempo_utilizado_pabellon_horas,
    "horas_laborales_por_anio": horas_laborales,
    "pabellones_desg": cantidad_de_pabellones_necesarios,
    "pabellones": suma_total_pabellones,
    # "tiempo_ut_emergencia": tiempo_utilizado_pabellon_complicaciones_horas,
    # "pabellones_desg_emergencia": pabellones_por_complicaciones,
    # "pabellones_emergencia": suma_total_pabellones_complicaciones,
}

In [136]:
with pd.ExcelWriter("../data/interim/2.0_estimacion_pabellones_RDR.xlsx") as file:
    for nombre_hoja, df_a_guardar in a_guardar.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)